In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
from fpdf import FPDF

# Ensure the outputs directory exists
output_dir = '../outputs'
os.makedirs(output_dir, exist_ok=True)

# Load the datasets
customers_df = pd.read_csv('../data/Customers.csv')
products_df = pd.read_csv('../data/Products.csv')
transactions_df = pd.read_csv('../data/Transactions.csv')

# Check columns of each DataFrame
print("Customers DataFrame columns:", customers_df.columns)
print("Products DataFrame columns:", products_df.columns)
print("Transactions DataFrame columns:", transactions_df.columns)

# Merge datasets
merged_df = transactions_df.merge(customers_df, on='CustomerID').merge(products_df, on='ProductID')

# Check columns of the merged DataFrame
print("Merged DataFrame columns:", merged_df.columns)

# Feature engineering and scaling
# Ensure the column names exist in the merged DataFrame
required_columns = ['Price', 'Quantity', 'TotalValue']
missing_columns = [col for col in required_columns if col not in merged_df.columns]
if missing_columns:
    print(f"Missing columns in merged DataFrame: {missing_columns}")
else:
    features = merged_df[required_columns]
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Clustering
    kmeans = KMeans(n_clusters=5)
    clusters = kmeans.fit_predict(scaled_features)
    merged_df['Cluster'] = clusters

    # Evaluation
    db_index = davies_bouldin_score(scaled_features, clusters)
    print(f'Davies-Bouldin Index: {db_index}')

    # Visualize clusters
    plt.scatter(scaled_features[:, 0], scaled_features[:, 1], c=clusters, cmap='viridis')
    plt.xlabel('Feature1')
    plt.ylabel('Feature2')
    plt.title('Customer Segments')
    plt.savefig(os.path.join(output_dir, 'customer_segments.png'))
    plt.show()

    # Save the clustering report to a PDF
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Clustering Report", ln=True, align='C')
    pdf.cell(200, 10, txt=f"Davies-Bouldin Index: {db_index}", ln=True, align='L')
    
    # Save the PDF
    pdf_output_path = os.path.join(output_dir, "FirstName_LastName_Clustering.pdf")
    pdf.output(pdf_output_path)
    print(f"PDF saved to {pdf_output_path}")

Customers DataFrame columns: Index(['CustomerID', 'CustomerName', 'Region', 'SignupDate'], dtype='object')
Products DataFrame columns: Index(['ProductID', 'ProductName', 'Category', 'Price'], dtype='object')
Transactions DataFrame columns: Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price'],
      dtype='object')
Merged DataFrame columns: Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Price_y'],
      dtype='object')
Missing columns in merged DataFrame: ['Price']
